# Leveraging Gen AI for SAT Prep - Prompt Engineering

We'll start with installing transformers, torch and accelerate libraries

In [ ]:
!pip install transformers
!pip install torch
!pip install accelerate>=0.26.0

In [2]:
import transformers
import torch
import accelerate
print(transformers.__version__)
print(torch.__version__)
print(accelerate.__version__)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.48.3
2.5.1
1.3.0


To run this notebook, you will need a Hugging Face (HF) token as Llama models are gated and require users to accept Meta’s usage terms. To get a token, you will have to provide your contact information in the HF model page (https://huggingface.co/meta-llama/Llama-3.1-70B-Instruct), accept the terms, and you will receive an email once the access has been approved. Followed by that, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab by navigating to the "Secrets" tab in the left panel and creating a new secret named "HF_TOKEN" and paste your Hugging Face token as the value. Restart you session thereafer.

We use Llama 3.1-8B model due to resource restrictions. If you have a powerful machine, you can leverage the 70B model.

In [3]:
#model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"
model_id="meta-llama/Meta-Llama-3-8B-Instruct"

### SAT Vocabulary Dataset

I created a SAT vocabulary dataset containing 500+ words. You can download the csv from my Git Repository. I used a CSV fromat that can be converted to Hugging Face Datasets if need be.

In [4]:
import pandas as pd
import random
from random import randrange
vocab_df = pd.read_csv('sat_vocab.csv')
print("Sample word: {} ".format(vocab_df.head(5)))

Sample word:          word
0       Abate
1  Aberration
2       Abhor
3      Abject
4      Abjure 


### SAT Style Genre for Paragraph Generation

I created a Genre dataset that can be used to instruct the model to create context based on a given Genre.

In [5]:
genre_df = pd.read_csv('sat_genre.csv')
print("Sample genre: {} ".format(genre_df.head(5)))

Sample genre:                          genre
0    Emergence of Homo sapiens
1  Use of fire by early humans
2   Development of stone tools
3      Agricultural Revolution
4     Establishment of Jericho 


We have a test cases file that has random word and genre combination that we will use to evaluate differnet experiments to make the generation better.

In [69]:
import pandas
from pandas import DataFrame 
import os

# let's load the test cases file
#test_cases_file='eval_word_genre.csv'
test_cases_file='semantic_eval_1000_word_genre.csv'
test_cases = pandas.read_csv(test_cases_file)

test_cases.head(5)

,genre,word,similarity_score,definition,answer_choices
0,buddhist stupas,sublime,0.570303,awe,"sublime,manifest,vague,skeptic"
1,evolution of first tetrapods (four-limbed vert...,eclectic,0.539813,deriving ideas or style from a diverse range o...,"oblivious,tirade,vigilant,eclectic"
2,appearance of the first trees,inventive,0.513281,"imaginative, creative, able to think of new th...","castigate,taint,utilitarian,inventive"
3,emergence of earth's first atmosphere,viable,0.525630,capable of working or being successfully imple...,"stolid,abstain,belittle,viable"
4,alexander the great's conquests,fervor,0.524804,great warmth and intensity of feeling; She spo...,"belittle,vapid,fervor,vague"


## Download Llama Model
The following step will download the model weights and will take about 8 - 10 mins.

You will need Hugging Face token for downloading the weights. You can pass the token in the api or run "huggingface-cli login" and pass the token using a terminal window.

In [7]:
from transformers import LlamaForCausalLM, AutoModelForCausalLM, AutoTokenizer
import torch
from huggingface_hub import login

access_token="<include your Huggingface token here>"
login(token = access_token)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cache_dir="/home/ubuntu/Pragyan/model_cache"

model=AutoModelForCausalLM.from_pretrained(model_id, token=access_token, cache_dir=cache_dir).to(device)
#Use the line below if you are loading a finetuned model
#model=AutoModelForCausalLM.from_pretrained("<finetuned-model-location").to(device)
tokenizer= AutoTokenizer.from_pretrained(model_id, token=access_token, cache_dir=cache_dir)

2025-02-08 19:47:56.477392: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739044076.494944    6013 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739044076.500297    6013 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


cuda:0


Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.17s/it]


Inference Configuration - generating deterministic output, output logits, probabilities, etc.

In [28]:
from transformers import GenerationConfig
generation_config = GenerationConfig(
        # number of tokens to generate
        max_new_tokens=150,  
        # only choose from the top k most likely words
        top_k=50,  
        # Whether or not to use sampling ; use greedy decoding otherwise.
        do_sample=True,
        # parameter that controls the randomness or creativity of the generated text
        temperature=0.001, 
        # sets the pad tokens to whatever it is in the tokenizer
        pad_token_id=tokenizer.eos_token_id, 
        # output unnormalized outputs
        output_logits=True,
        # output the probabilities
        output_scores=True,   
        # passes hidden state along with output
        output_hidden_states=True,
        #returns output as a dict
        return_dict_in_generate=True,
        # reduce repetition
        #repetition_penalty=1.5
    )

print(generation_config)

GenerationConfig {
  "do_sample": true,
  "max_new_tokens": 150,
  "output_hidden_states": true,
  "output_logits": true,
  "output_scores": true,
  "pad_token_id": 128009,
  "return_dict_in_generate": true,
  "temperature": 0.001
}



Utility methods for preprocessing, running inference and post processing

In [49]:
import csv
import numpy as np

paragraph_length=150
replacement_mask='________________'

# function for running inference against the model
def run_inference(prompt, paragraph_length):
    inputs = tokenizer([prompt], return_tensors="pt").to(device)
    outputs=model.generate(**inputs, generation_config=generation_config)
    transition_scores = model.compute_transition_scores(outputs.sequences, outputs.scores, normalize_logits=True)  
    input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
    complete_text=''
    for t in outputs.sequences:
        complete_text += tokenizer.decode(t)
        
    generated_tokens = outputs.sequences[:,input_length:]
    generated_text = ''
    for t in generated_tokens:
        generated_text += tokenizer.decode(t)
        
    return [generated_text, generated_tokens, transition_scores, complete_text]

# Utility function for running experiment in a batch fashion
def run_experiment(input_prompt, test_cases_df, num_tests, select_random=False):
    output_df = DataFrame(columns=['word', 'genre', 'complete_response', 'generated_text','answer_choices','probability', 'full_word_tokenized'])
    counter = 0
    for index, row in test_cases_df.iterrows():
        word = ""
        genre = ""
        if select_random:
            word = (vocab_df['word'][randrange(vocab_df.shape[0])]).lower()
            genre = (genre_df['genre'][randrange(genre_df.shape[0])]).lower()
        else:
            word = row['word'].lower()
            genre = row['genre'].lower()
        
        print("Processing {}; word: {}; genre: {}".format(index+1, word, genre))
        final_prompt = input_prompt.format(genre, word)
        print("Prompt: {}".format(final_prompt))
        output = run_inference(final_prompt, paragraph_length)
        para = output[0]
        probability = ""
        answer_choices = ""
        if word in para:
            para = mask_word(word, para)
            answer_choices = get_answer_choices(word)
            probability = get_probability_score(output[1], output[2], word)
            counter += 1
        else:
            probability = "SAT word not included in the paragraph"
            answer_choices = "N/A"
            
        full_word_tokenized = False
        tokens_string = str(probability)
        if word in tokens_string:
            full_word_tokenized = True
            
        output_df = output_df.append({'word': word, 'genre': genre, 'complete_response': output[3], 'generated_text': para, 
                'answer_choices': answer_choices, 'probability':  probability, 'full_word_tokenized': full_word_tokenized}, ignore_index=True)
        
        if (index + 1) >= num_tests:
            break
    return output_df

# run output to a file
def write_output_to_file(df, output_file):
    # delete output file, if it exists
    if os.path.exists(output_file):
        os.remove(output_file)
    
    # write the results to file
    f = open(output_file,"w",newline="")
    df.to_csv(f)
    f.flush()
    f.close() 

# function for masking the SAT word from the generated paragraph
def mask_word(word, paragraph):
  return paragraph.replace(word, replacement_mask)

# function to create a list of answer choices that has the correct work and three incorrect word
def get_answer_choices(word):
  ans_choices = [word.lower()]
  for i in range(20):
    temp_ans = vocab_df['word'][randrange(vocab_df.shape[0])]
    if temp_ans not in ans_choices:
      ans_choices.append(temp_ans.lower())
    if(len(ans_choices) >= 4):
      break
  random.shuffle(ans_choices)
  return ans_choices

# function for formatting answer choices
def format_answer_choices(answer_choices):
    return "1/ {}, 2/ {}, 3/ {}, 4/ {}".format(answer_choices[0],answer_choices[1],answer_choices[2],answer_choices[3])
  
# function for selecting random word, genre combination and writing to tile
def write_word_genre_to_csv(file_name, count):
    with open(file_name, 'w', newline='') as csvfile:
        fieldnames = ['word', 'genre']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(count):
            writer.writerow({'word':vocab_df['word'][randrange(vocab_df.shape[0])],'genre':genre_df['genre'][randrange(genre_df.shape[0])]})
    write_word_genre_to_csv(file_name, 100)

# get word generation probability 
def get_probability_score(generated_tokens, transition_scores, word):
    prob = []
    for tok, score in zip(generated_tokens[0], transition_scores[0]):
        s = str(tokenizer.decode(tok)).strip()
        if s in word:
            prob.append(f"{tokenizer.decode(tok)}, {np.exp(score.numpy(force=True)):.2%}, {tok:5d} ")
    return prob

# post processing
def run_post_processing_logic(paragraph, word):
    # if more than one occurance of the word, mark the row as invalid
    if (paragraph.lower().count(replacement_mask))>1:
        return [False, paragraph, word]
    if not paragraph.endswith("."):
        parts = paragraph.split(".")
        # Remove the last part
        parts = parts[:-1]
        # Join the remaining parts back into a string
        paragraph = ".".join(parts)
        return [True, paragraph+".", word]
    # no processing needed
    return [True, paragraph, word]

    

## Experiment 1: Basic Prompt

In [27]:
zero_shot_output_file='output_basic.csv'
basic_prompt = "Generate a paragraph on {} with the word {} in it."
num_tests=1
df = run_experiment(basic_prompt, test_cases, num_tests, True)
print()
print("Generated Text: {}".format(df['generated_text'][0]))
print()
print("Answer Choices: {}".format(df['answer_choices'][0]))
print()
print("Word Generation Probability: {}".format(df['probability'][0]))

#uncomment the line below to write the output to file
#write_output_to_file(df, zero_shot_output_file)

Processing 1; word: ascend; genre: first successful organ transplants
Prompt: Generate a paragraph on first successful organ transplants with the word ascend in it.

Generated Text:  
The first successful organ transplant was performed by Dr. Joseph Murray in 1954. The transplant was a kidney transplant between identical twins, and it marked a major milestone in the field of organ transplantation. The surgery was a groundbreaking achievement, and it paved the way for future organ transplants. As the medical community began to ________________ to new heights, the number of successful transplants increased, and the lives of countless individuals were saved. The success of the first transplant also led to the development of new surgical techniques and the establishment of organ transplant programs around the world. Today, organ transplantation is a common and life-saving procedure, and it continues to ________________ to new heights, with thousands of transplants performed every year. 
Ge

## Experiment 2: Zero-shot Prompt Engineering

In [28]:
zero_shot_output_file='output_zero_shot.csv'
zero_shot='''Question: Generate a paragraph on The history of figure skating with the word deterimental in it. 
The Answer is: Insert Paragraph here 
Generate a paragraph on {} with the word {} in it.'''
num_tests=1
df = run_experiment(zero_shot, test_cases, num_tests, True)
print()
print("Generated Text: {}".format(df['generated_text'][0]))
print()
print("Answer Choices: {}".format(df['answer_choices'][0]))
print()
print("Word Generation Probability: {}".format(df['probability'][0]))

#uncomment the line below to write the output to file
#write_output_to_file(df, zero_shot_output_file)

Processing 1; word: unwarranted; genre: founding of the people's republic of china
Prompt: Question: Generate a paragraph on The history of figure skating with the word deterimental in it. 
The Answer is: Insert Paragraph here 
Generate a paragraph on founding of the people's republic of china with the word unwarranted in it.

Generated Text:  
The Answer is: The founding of the People's Republic of China on October 1, 1949, marked a significant turning point in the country's history. The establishment of the communist government led by Mao Zedong was met with ________________ resistance from the Nationalist Party, which had been in power since the fall of the Qing dynasty. The subsequent civil war resulted in the defeat of the Nationalist Party and the establishment of a single-party state. The founding of the People's Republic of China had a profound impact on the country's politics, economy, and society, shaping the course of its development for decades to come. 
Generate a paragrap

## Experiment 3: Few-shot Prompt Engineering

I use few-short prompting to teach the model the output format we would like. We add specific example of the desired output for the model to follow. This technique is called "few-shot prompting". 

We evaluate the validity of the test questions by checking if the tokenizer has the word and manually checking if the generate paragraph is meaningful, in other words the model is not halucinating. 

In [30]:
few_shot_output_file='output_few_shot_output.csv'
few_shot='''Question: Generate a paragraph on The history of figure skating with the word deterimental in it. 
The Answer is: Figure skating has a rich and storied history that spans over 4,000 years. The earliest evidence of 
figure skating dates back to the 12th century in Scandinavia, where it was a popular mode of transportation during the 
winter months. However, the sport as we know it today began to take shape in the 18th century, with the establishment of the 
first skating clubs in Europe. The introduction of artificial ice rinks in the late 19th century revolutionized the sport, 
allowing for more precise and controlled movements. Unfortunately, the rapid growth and commercialization of figure skating 
in the 20th century had a deterimental effect on the sport, leading to a focus on flashy jumps and spins over technical skill 
and art. 
Generate a paragraph on {} with the word {} in it.'''
num_tests=1
df = run_experiment(few_shot, test_cases, num_tests, True) 
print()
print("Generated Text: {}".format(df['generated_text'][0]))
print()
print("Answer Choices: {}".format(df['answer_choices'][0]))
print()
print("Word Generation Probability: {}".format(df['probability'][0]))

#uncomment the line below to write the output to file
#write_output_to_file(df, few_shot_output_file)

Processing 1; word: juxtapose; genre: appearance of primates
Prompt: Question: Generate a paragraph on The history of figure skating with the word deterimental in it. 
The Answer is: Figure skating has a rich and storied history that spans over 4,000 years. The earliest evidence of 
figure skating dates back to the 12th century in Scandinavia, where it was a popular mode of transportation during the 
winter months. However, the sport as we know it today began to take shape in the 18th century, with the establishment of the 
first skating clubs in Europe. The introduction of artificial ice rinks in the late 19th century revolutionized the sport, 
allowing for more precise and controlled movements. Unfortunately, the rapid growth and commercialization of figure skating 
in the 20th century had a deterimental effect on the sport, leading to a focus on flashy jumps and spins over technical skill 
and art. 
Generate a paragraph on appearance of primates with the word juxtapose in it.

Gener

## Experiment 4: Role based Prompt Engineering

In [31]:
role_based_output_file='output_role_based.csv'
role_based_shot='''You are a teacher writing a reasearch paper on a certain subject. Your task is to write a intro paragraph summarizing everything 
about the subject, however, you must use certain words.
Generate a paragraph on {} with the word {} in it.'''
num_tests=1
df = run_experiment(role_based_shot, test_cases, num_tests, True) 
print()
print("Generated Text: {}".format(df['generated_text'][0]))
print()
print("Answer Choices: {}".format(df['answer_choices'][0]))
print()
print("Word Generation Probability: {}".format(df['probability'][0]))

#uncomment the line below to write the output to file
#write_output_to_file(df, role_based_output_file)

Processing 1; word: benign; genre: founding of the mughal empire
Prompt: You are a teacher writing a reasearch paper on a certain subject. Your task is to write a intro paragraph summarizing everything 
about the subject, however, you must use certain words.
Generate a paragraph on founding of the mughal empire with the word benign in it.

Generated Text:  
The Mughal Empire, a vast and sprawling dominion that stretched from the snow-capped Himalayas to the scorching deserts of Gujarat, was founded by the ________________ and visionary Babur, a Central Asian ruler who brought with him a rich cultural heritage and a thirst for conquest. In 1526, Babur, a descendant of Timur and Genghis Khan, defeated the last ruler of the Delhi Sultanate, Ibrahim Lodi, at the Battle of Panipat, marking the beginning of the Mughal Empire's ascendance to power. With his military prowess and strategic genius, Babur established a strong foundation for the empire, which would go on to become one of the most 

## Experiment 5: Chain of Thought Prompt Engineering

In [52]:
cot_output_file='output_chain_of_thought.csv'
cot_shot='''You are a teacher writing a research paper on a certain subject. Your task is to write a intro paragraph summarizing everything 
about the subject, however, you must use certain words.
1. talk about the history of your subject
2. give specific examples
3. talk about nuances and discrepencys that stand out
4. give specific numbers
Generate a paragraph on {} with the word {} in it.'''
num_tests=2
df = run_experiment(cot_shot, test_cases, num_tests, True) 
print()
print("Generated Text: {}".format(df['generated_text'][0]))
print()
print("Answer Choices: {}".format(df['answer_choices'][0]))
print()
print("Word Generation Probability: {}".format(df['probability'][0]))

#uncomment the line below to write the output to file
#write_output_to_file(df, cot_output_file)

Processing 1; word: irresolute; genre: formation of the mediterranean sea
Prompt: You are a teacher writing a research paper on a certain subject. Your task is to write a intro paragraph summarizing everything 
about the subject, however, you must use certain words.
1. talk about the history of your subject
2. give specific examples
3. talk about nuances and discrepencys that stand out
4. give specific numbers
Generate a paragraph on formation of the mediterranean sea with the word irresolute in it.
Processing 2; word: arcane; genre: permian-triassic extinction event
Prompt: You are a teacher writing a research paper on a certain subject. Your task is to write a intro paragraph summarizing everything 
about the subject, however, you must use certain words.
1. talk about the history of your subject
2. give specific examples
3. talk about nuances and discrepencys that stand out
4. give specific numbers
Generate a paragraph on permian-triassic extinction event with the word arcane in it.


## Experiment 6: Retrieval Augmented Generation

In [70]:
# Utility function for running RAG experiment
def run_rag_experiment(output_df, test_cases_df, num_tests, select_random=False):
    counter = 0
    for index, row in test_cases_df.iterrows():
        if select_random:
            word = (vocab_df['word'][randrange(vocab_df.shape[0])]).lower()
            genre = (genre_df['genre'][randrange(genre_df.shape[0])]).lower()
        else:
            word = row['word'].lower()
            genre = row['genre'].lower()
            
        word = row['word'].lower()
        genre = row['genre'].lower()

        definition = row['definition'].lower()
        
        final_prompt='''Given the meaning of the word {}: {} Generate a paragraph on {} with the word {}. Make sure your paragraph is one single paragraph that is formally worded. If you are done generating the paragraph, stop. Make sure to use the given word "{}" as is and only use the word "{}" once.'''.format(word, definition, genre, word, word, word)
        print("Processing {}; word: {}; genre: {}".format(index+1, word, genre))
        print("Prompt: {}".format(final_prompt))
        output = run_inference(final_prompt, paragraph_length)
        para = output[0]
        probability = ""
        answer_choices = ""
        if word in para:
            para = mask_word(word, para)
            answer_choices = row['answer_choices']
            probability = get_probability_score(output[1], output[2], word)
            counter += 1
        else:
            probability = "SAT word not included in the paragraph"
            answer_choices = "N/A"

        post_process = run_post_processing_logic(para, word)
        is_valid= post_process[0]
        para = post_process[1]
            
        full_word_tokenized = False
        tokens_string = str(probability)
        if word in tokens_string:
            full_word_tokenized = True
            
        output_df = output_df.append({'word': word, 'genre': genre, 'complete_response': output[3], 'generated_text': para, 
                'answer_choices': answer_choices, 'probability':  probability, 'full_word_tokenized': full_word_tokenized, 
                'is_valid': is_valid}, ignore_index=True)
        
        if (index + 1) >= num_tests:
            break
    print("paragraphs with words {} out of {} rows".format(counter, output_df.shape[0]))
    return output_df

In [ ]:
rag_shot_output_file='output_rag_shot_output.csv'

num_tests=1000
df = DataFrame(columns=['word', 'genre', 'complete_response', 'generated_text','answer_choices','probability', 'full_word_tokenized'])
df = run_rag_experiment(df, test_cases, num_tests) 
print()
print("Complete Response: {}".format(df['complete_response'][0]))
print()
print("Generated Text: {}".format(df['generated_text'][0]))
print()
print("Answer Choices: {}".format(df['answer_choices'][0]))
print()
print("Word Generation Probability: {}".format(df['probability'][0]))

#uncomment the line below to write the output to file
write_output_to_file(df, rag_shot_output_file)